In [90]:
#Importo las librerias que se voy a utilizar.
import pandas as pd
from collections import Counter
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity


In [91]:
# Cargo el archivo CSV en un DataFrame para poder utilizarlo.
df = pd.read_csv('plataformas.csv',parse_dates=['date_added'])

<b>Crear 7 funciones para la FastApi:<b>



Película (sólo película, no serie, ni documentales, etc) con mayor duración según año, plataforma y tipo de duración. La función debe llamarse get_max_duration(year, platform, duration_type) y debe devolver sólo el string del nombre de la película.


In [92]:
def get_max_duration(anio: int, plataforma: str, dtype: str):
    
    # Me aseguro que si el usuario escribe en mayusculas se pase a minusculas.
    dtype = dtype.lower()
    platform = plataforma.lower()[0] #Aclaro indice en 0 para que tome la primer letra

    # Filtro por año de lanzamiento, plataforma aclarando que solo tenga en cuenta la primer letra de la columna id y que el tipo sea 'movie'.
    data_filtrada = df[(df['release_year'] == anio) & (df['id'].str.startswith(platform)) & (df['duration_type'] == dtype) & (df['type'] == 'movie')]
    
    # Busco la duración máxima dentro del conjunto de datos filtrados
    max_duration = data_filtrada['duration_int'].max()
    # Busco la fila que contiene la duración máxima dentro del conjunto de datos filtrados
    max_duration_row = data_filtrada.loc[data_filtrada['duration_int'] == max_duration]    
     
    #me aseguro que no este vacia para ser entregada.   
    if not max_duration_row.empty:
        # Si se encontró una fila con la duración máxima, se obtiene el nombre de la
        # película correspondiente (es decir, la película con la duración máxima)
        # extrayendo el valor de la columna 'title' de la fila.
        movie_name = max_duration_row['title'].iloc[0]
        
        return {'pelicula': movie_name}
    
    else:
        
        return {'No se encontraron peliculas que cumplan con los criterios de busqueda.'}

In [93]:
get_max_duration(2020,'NETFLIX','min')

{'pelicula': 'unbreakable kimmy schmidt: kimmy vs. the reverend'}

In [94]:
get_max_duration(2018,'amazon','min')


{'pelicula': 'soothing surf at del norte for sleep black screen'}

In [95]:
get_max_duration(2020,'disney','min')


{'pelicula': "dory's reef cam"}

In [96]:
get_max_duration(2020,'hulu','min')

{'pelicula': 'crock of gold: a few rounds with shane macgowan'}

--------------------------------------------------------------------------------------------------------------------------------------------------------


Cantidad de películas (sólo películas, no series, ni documentales, etc) según plataforma, con un puntaje mayor a XX en determinado año. La función debe llamarse get_score_count(platform, scored, year) y debe devolver un int, con el total de películas que cumplen lo solicitado.


In [97]:
def get_score_count(plataforma: str, scored: float, year: int):
      
    # Me aseguro que si el usuario escribe en mayus se pase a minusculas, en el caso de plataform solo me interesa la primer letra.
    platform = plataforma.lower()[0]#Aclaro indice en 0 para que tome la primer letra
        
    # Filtro por año de lanzamiento, plataforma aclarando que solo tenga en cuenta la primer letra de la columna id y que el tipo sea 'movie'.   
    data_filtrada = df[(df['id'].str.startswith(platform)) & (df['release_year'] == year) & (df['type'] == 'movie')]
    
    # Cuento la cantidad de películas que tienen el rating deseado
    count = data_filtrada[data_filtrada['score'] > scored]['id'].count()
    
    return {'plataforma': plataforma,
            'cantidad': int(count),
            'anio': year,
            'score': scored}

In [98]:
get_score_count('NetFlix',3.5, 2020)


{'plataforma': 'NetFlix', 'cantidad': 190, 'anio': 2020, 'score': 3.5}

In [99]:
get_score_count('amazon',3.5, 2020)


{'plataforma': 'amazon', 'cantidad': 267, 'anio': 2020, 'score': 3.5}

In [100]:
get_score_count('hulu',3.5, 2020)


{'plataforma': 'hulu', 'cantidad': 83, 'anio': 2020, 'score': 3.5}

In [101]:
get_score_count('disney',3.5, 2020)

{'plataforma': 'disney', 'cantidad': 34, 'anio': 2020, 'score': 3.5}

--------------------------------------------------------------------------------------------------------------------------------------------------------


Cantidad de películas (sólo películas, no series, ni documentales, etc) según plataforma. La función debe llamarse get_count_platform(platform) y debe devolver un int, con el número total de películas de esa plataforma. Las plataformas deben llamarse amazon, netflix, hulu, disney.


In [102]:
def get_count_platform(plataforma: str):
        
    # Me aseguro que si el usuario escribe en mayus se pase a minusculas, en el caso de plataforma solo me interesa la primer letra.
    platform = plataforma.lower()[0]#Aclaro indice en 0 para que tome la primer letra
    
    # Realizo el filtro para que cuente del dataframe solo la plataforma de la columna id y movie de la columna type.
    data_filtrada = df[(df['id'].str.startswith(platform)) & (df['type'] == 'movie')].shape[0]# Realizo el .shape en 0 para que me traiga solo la cantidad de filas que hay de ese filtro.    

    return {'plataforma': plataforma,
            'peliculas': data_filtrada}

In [103]:
get_count_platform('hulu')


{'plataforma': 'hulu', 'peliculas': 1484}

In [104]:
get_count_platform('amazon')


{'plataforma': 'amazon', 'peliculas': 7814}

In [105]:
get_count_platform('disney')


{'plataforma': 'disney', 'peliculas': 1052}

In [106]:
get_count_platform('netflix')

{'plataforma': 'netflix', 'peliculas': 6131}

--------------------------------------------------------------------------------------------------------------------------------------------------------


Actor que más se repite según plataforma y año. La función debe llamarse get_actor(platform, year) y debe devolver sólo el string con el nombre del actor que más se repite según la plataforma y el año dado.


In [107]:
def get_actor(plataforma: str, anio: int):    
    
    # Me aseguro que si el usuario escribe en mayus se pase a minusculas, en el caso de plataform solo me interesa la primer letra.
    platform = plataforma.lower()[0]#Aclaro indice en 0 para que tome la primer letra
    
    # Realizo el filtro para que cuente del dataframe solo la plataforma de la columna id y el añio.
    data_filtrada = df[(df['id'].str.startswith(platform)) & (df['release_year'] == anio)]   
 
    # En este paso, se utiliza la función dropna() de pandas para eliminar los valores NaN de la columna "cast" del DataFrame. 
    # Luego utilizo el método apply() para aplicar la función str() a cada valor de la columna "cast", convirtiendo los valores a cadenas de caracteres. 
    # Finalmente, se utiliza el método str.split(',') para dividir cada cadena de caracteres en una lista de actores utilizando la coma como separador
    # y un .strip para que extraiga cualquier valor que sea tipo digito(No enteros si no un .digit). 
    # El resultado es una serie de pandas que contiene una lista de actores para cada fila del DataFrame original, ignorando cualquier registro extraño.
    actores_por_fila = data_filtrada['cast'].dropna().apply(lambda x: [i.strip() for i in x.split(',') if not i.strip().isdigit()])
    
    # Cuento la cantidad de veces que aparece cada actor en todas las filas, utilizando la clase Counter de Python.
    contador_actores = Counter()
    for actores in actores_por_fila:
        # Actualizamos el contador de actores con un nuevo conjunto de actores.
        # Si el contador ya tenía un registro para un actor específico, se incrementa su
        # valor en 1. De lo contrario, se crea un nuevo registro para ese actor con un
        # valor inicial de 1.
        contador_actores.update(actores)

    # Encuentro el actor que aparece más veces utilizando la funcion most common devolviendo una lista de tuplas donde cada tupla contiene un actor 
    # y la cantidad de veces que aparece en todas las filas del DataFrame.
    actor_mas_repetido = contador_actores.most_common(1)# El 1 indica que se muestre 1 solo valor.
    
    #se verifica si la lista actor_mas_repetido no está vacía.
    if actor_mas_repetido:
        # se asigna a actor_mas_repetido el primer elemento de la primera tupla en la lista actor_mas_repetido, que contiene el actor que aparece más veces.
        actor_mas_repetido = actor_mas_repetido[0][0]
    else:
        return {'plataforma': plataforma,
                'anio': anio,
                'actor': "No hay datos disponibles",
                'apariciones': "No hay datos disponibles"}

    # Muestro el actor que aparece más veces y la cantidad de veces que aparece
    cantidad_actor_mas_repetido = contador_actores[actor_mas_repetido]
    return {'plataforma': plataforma,
            'anio': anio,
            'actor': actor_mas_repetido,
            'apariciones': cantidad_actor_mas_repetido}

In [108]:
get_actor('amazon',2021)


{'plataforma': 'amazon', 'anio': 2021, 'actor': 'om nom', 'apariciones': 10}

In [109]:
get_actor('hulu',2020)


{'plataforma': 'hulu',
 'anio': 2020,
 'actor': 'No hay datos disponibles',
 'apariciones': 'No hay datos disponibles'}

In [110]:
get_actor('netflix',2020)


{'plataforma': 'netflix',
 'anio': 2020,
 'actor': 'blossom chukwujekwu',
 'apariciones': 6}

In [111]:
get_actor('disney',2020)

{'plataforma': 'disney',
 'anio': 2020,
 'actor': 'daveed diggs',
 'apariciones': 3}

--------------------------------------------------------------------------------------------------------------------------------------------------------


La cantidad de contenidos/productos (todo lo disponible en streaming) que se publicó por país y año. La función debe llamarse prod_per_county(tipo,pais,anio) deberia devolver el tipo de contenido (pelicula,serie) por pais y año en un diccionario con las variables llamadas 'pais' (nombre del pais), 'anio' (año), 'pelicula' (tipo de contenido).


In [112]:
def prod_per_county(tipo: str, pais: str, anio: int):   
    
    # Realizo un filtro para que busque por tipo, año de lanzamiento y que el campo "country" contenga el país.
    data_filtrada = df[(df['type'] == tipo) & (df['release_year'] == anio) & (df['country'].str.contains(pais))]
    # Creo un diccionario vacío para contar la cantidad de películas por país
    count_por_pais = {}

    # Recorro cada fila del DataFrame y cada lista de países en ella
    for countries in data_filtrada['country']:
        
        # Separo los países de la lista y los recorro
        for country in countries.split(','):
            
            # Elimino los espacios en blanco al inicio y al final del país
            country = country.strip()
            
            # Verifico si el país ya existe en el diccionario, si no es así lo agrego y le asigno un valor de 0
            if country not in count_por_pais:
                count_por_pais[country] = 0
                
            # Incremento en 1 la cantidad de películas para ese país en el diccionario
            count_por_pais[country] += 1 
               
    # Obtener el recuento de películas para el país especificado
    respuesta = count_por_pais.get(pais, 0)
    
    return {'pais': pais,
            'anio': anio,            
            'peliculas': respuesta}

In [113]:
prod_per_county('movie','united arab emirates' , 2014)

{'pais': 'united arab emirates', 'anio': 2014, 'peliculas': 5}

--------------------------------------------------------------------------------------------------------------------------------------------------------


La cantidad total de contenidos/productos (todo lo disponible en streaming, series, documentales, peliculas, etc) según el rating de audiencia dado (para que publico fue clasificada la pelicula). La función debe llamarse get_contents(rating) y debe devolver el numero total de contenido con ese rating de audiencias.

In [114]:
def get_contents(rating: str):       
    
    #Realizo un .value_counts de rating en base al rating que pongan.    
    respuesta =  df['rating'].value_counts()[rating]

    return {'rating': rating,
            'contenido': int(respuesta)}

In [115]:
get_contents('g')

{'rating': 'g', 'contenido': 1269}

--------------------------------------------------------------------------------------------------------------------------------------------------------

Sistema de recomendación:

Una vez que toda la data es consumible por la API, está lista para consumir por los departamentos de Analytics y Machine Learning, y nuestro EDA nos permite entender bien los datos a los que tenemos acceso, es hora de entrenar nuestro modelo de machine learning para armar un sistema de recomendación de películas. Éste consiste en recomendar películas a los usuarios basándose en películas similares, por lo que se debe encontrar la similitud de puntuación entre esa película y el resto de películas, se ordenarán según el score y devolverá una lista de Python con 5 valores, cada uno siendo el string del nombre de las películas con mayor puntaje, en orden descendente. Debe ser deployado como una función adicional de la API anterior y debe llamarse get_recommendation(titulo: str).

<b>Version A<b>

In [116]:
# Creo una instancia de TfidfVectorizer con stop words en inglés
tfidf = TfidfVectorizer(stop_words='english')

# Convierto los títulos en una matriz TF-IDF
tfidf_matrix = tfidf.fit_transform(df['title'])


def get_recommendations(title: str):
    #Me aseguro que si escriben en mayuscula llevarlo a minuscula.
    title = title.lower()
    # Creo una serie con los índices de las películas y sus títulos correspondientes
    indices = pd.Series(df.index, index=df['title'])
    # Obtengo el índice de la película que se quiere buscar
    idx = indices[title]

    # Calculo la similitud coseno entre la película buscada y todas las demás
    cosine_sim = cosine_similarity(tfidf_matrix[idx], tfidf_matrix)
    # Obtengo las similitudes y sus índices correspondientes
    sim_scores = list(enumerate(cosine_sim[0]))
    # Ordeno las similitudes de mayor a menor
    sim_scores = sorted(sim_scores, key=lambda x: x[1], reverse=True)
    # Tomo las 5 películas más similares (excluyendo la misma película buscada)
    sim_scores = sim_scores[1:6]
    # Obtengo los índices de las películas más similares
    movie_indices = [i[0] for i in sim_scores]
    # Obtengo los títulos de las películas más similares y los retornamos como una lista
    respuesta = df['title'].iloc[movie_indices].tolist()
    return {'recomendacion': respuesta}


In [117]:
get_recommendations('the grand seduction')

{'recomendacion': ['the grand',
  'five grand',
  'grand hotel',
  'the grand tour',
  'grand army']}

In [118]:
get_recommendations('tarzan')

{'recomendacion': ['tarzan',
  'tarzan',
  'tarzan 2',
  'tarzan ii',
  'come on, tarzan']}

<b>Version B<b>

In [119]:
def get_recommendationsB(title: str):
           
    #Me aseguro que si escriben en mayuscula llevarlo a minuscula.
    title = title.lower()    
    
    # Creo una instancia de TfidfVectorizer.
    vectorizer = TfidfVectorizer()
    
    # Aplico el vectorizador a la columna 'listed_in' para obtener la matriz de características
    tfidf_matrix = vectorizer.fit_transform(df['listed_in'])
    
    # Calculo la matriz de similitud de coseno
    cosine_sim = cosine_similarity(tfidf_matrix)
    
    # Obtengo el índice de la película que coincide con el título proporcionado
    idx = df.index[df['title'] == title.lower()].tolist()[0]

    # Obtengo las puntuaciones de similitud de coseno de la película con todas las demás películas
    sim_scores = list(enumerate(cosine_sim[idx]))

    # Ordeno las películas según las puntuaciones de similitud
    sim_scores = sorted(sim_scores, key=lambda x: x[1], reverse=True)

    # Obtengo las cinco películas más similares, excluyendo la película de entrada
    sim_scores = [i for i in sim_scores if df.index[i[0]] != idx]
    sim_scores = sim_scores[:5]

    # Obtengo los títulos de las cinco películas más similares
    recomendaciones = df.iloc[[i[0] for i in sim_scores]]['title'].tolist()

    return {'recomendacion': recomendaciones}

In [120]:
get_recommendationsB('the grand seduction')

{'recomendacion': ["summer '03",
  'z: the beginning of everything',
  'you, me and him',
  'walter',
  'waiting on mary']}

In [121]:
get_recommendationsB('tarzan')

{'recomendacion': ['on happiness road',
  'invincible',
  'metal skin panic madox-1',
  'series before 1c onboarding - 2',
  'nuttiest nutcracker']}

VERSION C

In [122]:
def get_recommendationsC(title: str):
    
    #Me aseguro que si escriben en mayuscula llevarlo a minuscula.
    title = title.lower()
    
    #Leo el ccsv que utilizo.
    df_corr = pd.read_csv('ML.csv')
    
    # Creo el vectorizador TF-IDF
    vectorizer = TfidfVectorizer()
    
    # Aplico el vectorizador a la columna 'listed_in' para obtener la matriz de características
    tfidf_matrix = vectorizer.fit_transform(df_corr['title'])
    
    # Calculo la matriz de similitud de coseno
    cosine_sim = cosine_similarity(tfidf_matrix) 
    idx = df_corr.index[df_corr['title'] == title.lower()].tolist()[0]
    
    # Obtengo las puntuaciones de similitud de coseno de la película con todas las demás películas
    sim_scores = list(enumerate(cosine_sim[idx]))
    
    # Ordeno las películas según las puntuaciones de similitud
    sim_scores = sorted(sim_scores, key=lambda x: x[1], reverse=True)
    
    # Obtengo las cinco películas más similares, excluyendo la película de entrada
    sim_scores = [i for i in sim_scores if df_corr.index[i[0]] != idx]
    sim_scores = sim_scores[:5]
    
    # Obtengo los títulos de las cinco películas más similares
    recomendaciones = df_corr.iloc[[i[0] for i in sim_scores]]['title'].tolist()

    return {'recomendacion': recomendaciones}

In [123]:
get_recommendationsC('the grand seduction')

{'recomendacion': ['the grand',
  'five grand',
  'grand isle',
  'winged seduction: birds of paradise',
  'into the grand canyon']}

In [124]:
get_recommendationsC('tarzan')

{'recomendacion': ['tarzan',
  'tarzan',
  'tarzan 2',
  'tarzan ii',
  'tarzan & jane']}

VERSION D

In [125]:
df_corr = pd.read_csv('ML.csv')

In [126]:
def get_recommendationsD(title: str):  
    #Me aseguro que si escriben en mayuscula llevarlo a minuscula.
    title = title.lower()
      
    # Creo el vectorizador TF-IDF
    tfidf = TfidfVectorizer(stop_words='english')
    
    # Aplico el vectorizador a la columna para obtener la matriz de características.
    tfidf_matrix = tfidf.fit_transform(df_corr['categorias'])
    
    # Obtengo el índice de la película que coincide con el título proporcionado
    idx = df_corr.index[df_corr['title'] == title.lower()].tolist()[0]
    
    # Calculo la matriz de similitud de coseno
    cosine_sim = cosine_similarity(tfidf_matrix[idx], tfidf_matrix)
    
    # Obtengo las puntuaciones de similitud de coseno de la película con todas las demás películas
    sim_scores = list(enumerate(cosine_sim[0]))
    
    # Ordeno las películas según las puntuaciones de similitud
    sim_scores = sorted(sim_scores, key=lambda x: x[1], reverse=True)
    
    # Obtengo las cinco películas más similares, excluyendo la película de entrada
    sim_scores = [i for i in sim_scores if i[0] != idx]
    sim_scores = sim_scores[:5]
    
    # Obtengo los títulos de las cinco películas más similares
    recomendaciones = df_corr.iloc[[i[0] for i in sim_scores]]['title'].tolist()
    
 
    return {'recomendacion': recomendaciones} 

In [127]:
get_recommendationsD('the grand seduction')

{'recomendacion': ['take care good night',
  'grease live!',
  "david's mother",
  'take care',
  "summer '03"]}

In [128]:
get_recommendationsD('tarzan')

{'recomendacion': ['stay tuned',
  'technotise: edit & i',
  'on happiness road',
  'the congress',
  'shrek 2']}

VERSION FINAL

In [129]:
def get_recomendation(title: str):    
    
    #Me aseguro que si escriben en mayuscula llevarlo a minuscula.
    title = title.lower()
    
    # Creo el vectorizador TF-IDF
    tfidf = TfidfVectorizer(stop_words='english')
    
    # Aplico el vectorizador a la columna para obtener la matriz de características.
    tfidf_matrix = tfidf.fit_transform(df_corr['categorias'])
    
    # Obtengo el índice de la película que coincide con el título proporcionado
    idx = df_corr.index[df_corr['title'] == title.lower()].tolist()[0]
    
    # Calculo la matriz de similitud de coseno
    cosine_sim = cosine_similarity(tfidf_matrix[idx], tfidf_matrix)
    
    # Obtengo las puntuaciones de similitud de coseno de la película con todas las demás películas
    sim_scores = list(enumerate(cosine_sim[0]))
    
    # Ordeno las películas según las puntuaciones de similitud
    sim_scores = sorted(sim_scores, key=lambda x: x[1], reverse=True)
    
    # Obtengo las películas similares con la mejor puntuación de score
    sim_scores = [i for i in sim_scores if i[0] != idx]
    sim_scores = sorted(sim_scores, key=lambda x: df_corr['score'].iloc[x[0]], reverse=True)[:5]
    
    # Obtengo los títulos de las películas seleccionadas
    respuesta = df_corr.iloc[[i[0] for i in sim_scores]]['title'].tolist()
    
    
    return {'recomendacion': respuesta}

In [130]:
get_recomendation('the grand seduction')

{'recomendacion': ['take care good night',
  'grease live!',
  "david's mother",
  'take care',
  "summer '03"]}

In [131]:
get_recomendation('tarzan')

{'recomendacion': ['stay tuned',
  'technotise: edit & i',
  'on happiness road',
  'the congress',
  'shrek 2']}